In [1]:
import os
import sys

from MaxText.globals import MAXTEXT_REPO_ROOT

# Add the project root to the system path if it's not already there
if MAXTEXT_REPO_ROOT not in sys.path:
  sys.path.insert(0, MAXTEXT_REPO_ROOT)
  print(f"Added '{MAXTEXT_REPO_ROOT}' to sys.path")

Added '/home/mazumdera/maxtext' to sys.path


In [ ]:
import MaxText as mt
from MaxText import pyconfig
from MaxText import maxtext_utils
import numpy as np
from MaxText.input_pipeline import _input_pipeline_utils
import os
from MaxText import max_logging
from MaxText import common_types
import jax
from maxtext import inference_utils

2025-06-18 21:34:12.489183: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750282452.508183 1726814 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750282452.513660 1726814 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750282452.528073 1726814 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750282452.528091 1726814 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750282452.528093 1726814 computation_placer.cc:177] computation placer alr

In [ ]:
from MaxText.globals import MAXTEXT_PKG_DIR

config = pyconfig.initialize(
    [os.path.join(MAXTEXT_PKG_DIR, "decode.py"), os.path.join(MAXTEXT_PKG_DIR, "configs", "base.yml")],
    per_device_batch_size=1.0,
    run_name="test",
    enable_checkpointing=False,
    base_num_decoder_layers=2,
    max_target_length=4,
    base_emb_dim=256,
    base_num_query_heads=2,
    base_num_kv_heads=2,
    max_prefill_predict_length=4,
    # tokenizer_path="assets/tokenizers/llama3.1-tokenizer/",
    # model_name="llama3.1-7b",
)

model = mt.from_config(config)
mesh = model.mesh
init_rng = jax.random.PRNGKey(config.init_weights_seed)
state, _ = maxtext_utils.setup_decode_state(model, config, init_rng, mesh, None)

Updating keys from env and command line: ['run_name', 'enable_checkpointing', 'base_emb_dim', 'base_num_query_heads', 'base_num_kv_heads', 'base_num_decoder_layers', 'per_device_batch_size', 'max_target_length', 'max_prefill_predict_length']
Running Model: default
Updating keys from model: []


Not using emergency checkpoint, ignoring local_checkpoint_directory, local_checkpoint_period, use_replicator_service and replicator_backup_interval_minutes
dataset_type set to tfds, will use keys['dataset_path']='' and keys['dataset_name']='c4/en:3.0.1'
Config param activations_in_float32: False
Config param adam_b1: 0.9
Config param adam_b2: 0.95
Config param adam_eps: 1e-08
Config param adam_eps_root: 0.0
Config param adam_weight_decay: 0.1
Config param add_bos: True
Config param add_eos: True
Config param allow_split_physical_axes: False
Config param ar_cache_axis_order: 1,2,0,3
Config param async_checkpointing: True
Config param attention: autoselected
Config param attention_type: global
Config param attn_logits_soft_cap: None
Config param autoregressive_decode_assert: 
Config param base_emb_dim: 256
Config param base_mlp_dim: 7168
Config param base_moe_mlp_dim: 7168
Config param base_num_decoder_layers: 2
Config param base_num_kv_heads: 2
Config param base_num_query_heads: 2
Confi

Config param inference_microbenchmark_loop_iters: 10
Config param inference_microbenchmark_num_samples: [1, 2, 3, 4, 5]
Config param inference_microbenchmark_prefill_lengths: 64,128,256,512,1024
Config param inference_microbenchmark_stages: prefill,generate
Config param inference_server: MaxtextInterleavedServer
Config param inhomogeneous_layer_cycle_interval: 1
Config param init_weights_seed: 0
Config param input_data_sharding_logical_axes: ['activation_embed_and_logits_batch', 'activation_norm_length']
Config param interleave_moe_layer_step: 1
Config param intermediate_size_for_vit: 5632
Config param jax_cache_dir: ~/jax_cache
Config param jax_debug_log_modules: 
Config param jax_distributed_initialization_timeout: 300
Config param jax_profiler_port: 9999
Config param key_proj: remat
Config param kv_lora_rank: 512
Config param kv_quant_axis: heads_and_dkv
Config param kv_quant_dtype: int8
Config param learning_rate: 3e-05
Config param learning_rate_schedule_steps: 150001
Config param

NameError: name 'global_store' is not defined

In [ ]:
from MaxText.globals import MAXTEXT_ASSETS_ROOT

source_tokenizer = _input_pipeline_utils.get_tokenizer(
    os.path.join(MAXTEXT_ASSETS_ROOT, "tokenizers", "tokenizer_llama3.tiktoken"),
    "tiktoken",
    add_bos=True,
    add_eos=False,
)


# TODO: @mazumdera: any way to geto segment and position ids like HF tokenizer gives us?
input_ids = source_tokenizer.encode(config.prompt)  # .numpy()
ids = np.asarray(input_ids, dtype=np.int32)
s = (config.global_batch_size_to_train_on, config.max_target_length)
decoder_segment_ids = np.zeros(s) + common_types.DECODING_ACTIVE_SEQUENCE_INDICATOR
decoder_positions = np.stack(
    [np.arange(config.max_target_length, dtype=np.int32) for _ in range(config.global_batch_size_to_train_on)]
)

# TODO: @mazumdera: simplify this config.global_batch_size_to_train_on=1
ids = np.stack([ids for _ in range(config.global_batch_size_to_train_on)])
max_logging.log(
    f"input_ids={input_ids}, ids={ids}, decoder_segment_ids = {decoder_segment_ids}, decoder_positions= {decoder_positions}"
)

Tokenizer path: /home/mazumdera/maxtext/assets/tokenizer_llama3.tiktoken
Reloaded tiktoken model from /home/mazumdera/maxtext/assets/tokenizer_llama3.tiktoken
#words: 128256 - BOS ID: 128000 - EOS ID: 128001
input_ids=[128000, 40, 3021, 311], ids=[[128000     40   3021    311]], decoder_segment_ids = [[1. 1. 1. 1.]], decoder_positions= [[0 1 2 3]]


In [ ]:
import jax

!export TPU_LIBRARY_PATH=/home/mazumdera/.local/lib/python3.10/site-packages/libtpu/libtpu.so

jax.devices()

[CpuDevice(id=0)]

In [ ]:
!ls /home/mazumdera/.local/lib/python3.10/site-packages/libtpu/libtpu.so

/home/mazumdera/.local/lib/python3.10/site-packages/libtpu/libtpu.so


In [ ]:
import jax.experimental.multihost_utils

full_train_logits = model.apply(
    state.params,
    ids,
    decoder_positions,
    decoder_segment_ids,
    enable_dropout=False,
    rngs={"aqt": init_rng},
)
full_train_logits = jax.experimental.multihost_utils.process_allgather(full_train_logits)
max_logging.log(f"{full_train_logits[0, 0, :]=}")

full_train_logits[0, 0, :]=array([[ 0.6484375 , -1.09375   , -1.3359375 , ...,  0.0177002 ,
        -0.8984375 , -0.57421875],
       [ 0.8125    , -0.53125   , -0.3125    , ...,  1.34375   ,
         1.078125  , -1.3828125 ],
       [ 0.6171875 , -2.        , -2.0625    , ...,  0.13867188,
        -0.9375    , -0.796875  ],
       [-0.27734375, -1.3203125 , -0.765625  , ...,  1.1171875 ,
        -0.26953125,  0.4296875 ]], dtype=float32)


In [ ]:
selected_logits = jax.lax.dynamic_slice(
    full_train_logits, (0, 0, full_train_logits.shape[2] - 1, 0), (1, 1, 1, full_train_logits.shape[3])
)

In [ ]:
init_rng, new_rng = jax.random.split(init_rng)
first_generated_token = inference_utils.sampling(
    selected_logits,
    new_rng,
    config.decode_sampling_strategy,
    topk=config.decode_sampling_top_k,
    nucleus_topp=config.decode_sampling_nucleus_p,
    temperature=config.decode_sampling_temperature,
)

In [ ]:
first_generated_token.item()

26831

In [ ]:
source_tokenizer.decode([first_generated_token.item()])

'-ad'